# [LRU Cache challenge from LeetCode](https://leetcode.com/problems/lru-cache/)

Design and implement a data structure for Least Recently Used (LRU) cache. It should support the following operations: `get` and `put`.

`get(key)` - Get the value (will always be positive) of the key if the key exists in the cache, otherwise return -1.

`put(key, value)` - Set or insert the value if the key is not already present. When the cache reached its capacity, it should invalidate the least recently used item before inserting a new item.

### Follow up:
Could you do both operations in O(1) time complexity?

### Better solutions:

I struggled a bit with this one trying to implement in O(1) time.  My solution is O(1) when you amortize cleaning step, but [there's a better solution mixing a doubly linked list and hash table.](https://leetcode.com/problems/lru-cache/)  I'm impressed with how clean and well laid out that solution is.

In [ ]:
from collections import deque

# Your LRUCache object will be instantiated and called as such:
# obj = LRUCache(capacity)
# param_1 = obj.get(key)
# obj.put(key,value)


class CacheElement:
    def __init__(self, age, key, value, status):
        self.age = age
        self.key = key
        self.value = value
        self.status = status
        
        
    def __repr__(self):
        return '(CacheElement: age = {}, key = {}, val = {}, status = {})'.format(
            self.age, self.key, self.value, self.status)

    
    def __str__(self):
        return '(CacheElement: age = {}, key = {}, val = {}, status = {})'.format(
            self.age, self.key, self.value, self.status)

    
class LRUCache:

    def __init__(self, capacity):
        """
        :type capacity: int
        """
        self.capacity = capacity
        self.size = 0
        self.min_age = 0      # age of min element, removed or active
        self.max_age = -1     # age of the oldest element
        self.cache = {}       # retrieves 'active' elem location by key
        self.deque = deque()  # stores elements by age order
        
        
    def clean_removed(self):
        while self.deque[0].status == 'removed':
            self.deque.popleft()
            self.min_age += 1
        

    def get(self, key):
        """
        :type key: int
        :rtype: int
        """
        # retrieve data from age_cache, return None if not present
        res_age = self.cache.get(key)
        
        if res_age is None:
            return -1
        
        res = self.deque[res_age - self.min_age]
        
        res.status = 'removed'
        self.max_age += 1
        new_elem = CacheElement(self.max_age, res.key, res.value, 'active')
        self.deque.append(new_elem)
        self.cache[key] = self.max_age
        
        self.clean_removed()
            
        return res.value
        

    def put(self, key, value):
        """
        :type key: int
        :type value: int
        :rtype: void
        """
        
        # create elem, increasing max_age by 1, increasing size by 1, add to end of deque
        self.max_age += 1
        new_elem = CacheElement(self.max_age, key, value, 'active')
        self.size += 1
        self.deque.append(new_elem)
        
        # if key already in cache, set deque location to 'removed', decrease size by 1
        previous_location = self.cache.get(key)
        
        if previous_location is not None:
            self.deque[previous_location - self.min_age].status = 'removed'
            self.size -= 1
        
        self.cache[key] = self.max_age
        
        self.clean_removed()
                
        # if size > capacity, clean, remove first element, delete it from cache, clean
        if self.size > self.capacity:
            to_delete = self.deque.popleft().key
            del self.cache[to_delete]
            self.min_age += 1
            self.size -= 1
            self.clean_removed()        
        
    
    def __repr__(self):
        s = '\n==== LRU Cache ====\n'
        s += 'capacity = {}, size = {}, min_age = {}, max_age ={}'.format(
            self.capacity, self.size, self.min_age, self.max_age)
        s += '\ncache = {}\ndeque = {}\n==== End LRU Cache ====\n'.format(
            self.cache, self.deque)
        
        return s
    
    def __str__(self):
        return self.__repr__()

# Improved implementation

Implementing the improved approach from the problem discussion.

In [13]:
class DLLNode:
    
    def __init__(self, data):
        self.data = data
        self.prev = None
        self.next = None
        
        
    def __repr__(self):
        return '-{}-'.format(self.data)
    
    
    def __str__(self):
        return self.__repr__()

        
class LRUCache_2:
    
    def __init__(self, capacity):
        self.capacity = capacity
        self.dict = {}  # associates key value to a node
        self.head = DLLNode(0)
        self.tail = DLLNode(1)
        self.head.next = self.tail
        self.tail.prev = self.head
        
        
    def _remove(self, node):
        previous = node.prev
        nextnode = node.next
        previous.next = nextnode
        nextnode.prev = previous
        
        
    def _add(self, node):
        previous = self.tail.prev
        previous.next = node
        node.prev = previous
        self.tail.prev = node
        node.next = self.tail
        
        
    def __repr__(self):
        s = 'head --'
        
        node = self.head.next
        
        while node != self.tail:
            s += str(node)
            node = node.next
        
        s += '-- tail'
        
        return s
    
    
    def __str__(self):
        return self.__repr__()

In [14]:
lru = LRUCache_2(3)
lru._add(DLLNode('b'))
lru._add(DLLNode((3,4)))
lru._add(DLLNode({'a': 2}))
print(lru)
lru._remove(lru.head.next)
print(lru)

head ---b--(3, 4)--{'a': 2}--- tail
head ---(3, 4)--{'a': 2}--- tail


In [1]:
def run_LRUCache_case(ops_data, input_data):
    res = []
    lru = LRUCache(capacity = input_data[0][0])
    res.append(None)
    
    for i in range(1, len(ops_data)):
        if ops_data[i] == 'get':
            res.append(lru.get(input_data[i][0]))
                
        elif ops_data[i] == 'put':
            res.append(None)
            lru.put(*input_data[i])
            
    return res

In [2]:
import unittest


class lru_tests(unittest.TestCase):
    
    def test_known_results(self):
        ops_data = ["LRUCache","put","put","get","put","get","put","get","get","get"]
        input_data = [[2],[1,1],[2,2],[1],[3,3],[2],[4,4],[1],[3],[4]]
        expected_output = [None,None,None,1,None,-1,None,-1,3,4]
    
        self.assertEqual(run_LRUCache_case(ops_data, input_data), expected_output)
        
        ops_data = ["LRUCache","put","put","get","get","put","get","get","get"]
        input_data = [[2],[2,1],[3,2],[3],[2],[4,3],[2],[3],[4]]
        expected_output = [None, None, None, 2, 1, None, 1, -1, 3]
        
        self.assertEqual(run_LRUCache_case(ops_data, input_data), expected_output)
        

if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK
